# Character Recurrent Neural Network
- Mimicing Shakespeare's writing style
- Gated Recurrent Unit(GRU)

![alt text](./GRU.png)

## 1. Settings
### 1) Import required libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
import unidecode
import string
import random
import re
import time, math

## 2) Hyperparameter

In [3]:
num_epochs = 20000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

## 2. Data
### 1) Prepare characters

In [4]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### 2) Get text data

In [5]:
file = unidecode.unidecode(open('./data/shakespeare.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


## 3. Functions for text processing
### 1) Random Chunk

In [6]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

larence' death?
Before I be convict by course of law,
To threaten me with death is most unlawful.
I charge you, as you hope to have redemption
By Christ's dear blood shed for our grievous sins,
That yo


### 2) Character to tensor

In [7]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor).cuda()

print(char_tensor('ABCdef'))

Variable containing:
 36
 37
 38
 13
 14
 15
[torch.cuda.LongTensor of size 6 (GPU 0)]



### 3) Chunk into input & label

In [8]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## 3. Model & Optimizer
### 1) Model

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.GRU(hidden_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda()
        return hidden
    
model = RNN(n_characters, hidden_size, n_characters, num_layers).cuda()

In [10]:
inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())

out,hidden = model(inp,hidden)
print(out.size())

Variable containing:
 36
[torch.cuda.LongTensor of size 1 (GPU 0)]

torch.Size([1, 1, 100])
torch.Size([1, 100])


### 2) Loss & Optimizer

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

### 3) Test function

In [ ]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

## 4. Train

In [ ]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j]
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 Variable containing:
 4.6094
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bC$#k9[VD0\Da(1XyH[q$D`4_utu*5J*
=|;EDY'V
cJ2S3\`>lmA,bL#<oii9^'Sc>BcU\H
*Aw3>4!`RFZ<ks_z`HEm_I[e3n6ds`tZND/]	Dab{w	9k#i|O%/d<z(*aANd+WSJ2O{3im7[nC]K{F



 Variable containing:
 2.5614
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bP.y pdakO had at iare coet pa pas ses pine conn hkisisthan oo yof dores, nire he uicarl smeing tond oures eme akvest gang this murs hoon asat
Sou thel gour he lo houc, arer th in noungelordourd you, y



 Variable containing:
 2.2744
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bug; ford wise there.

QSZATEA.
AWDKLO:
And herere of it his he I parliog conl and'd it dor tony sith radin to bourg wings hore the mnoll patint orthe-in of ort, thame ard xull donwem wind with can tho



 Variable containing:
 2.1945
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bvese upase food sherer not in my he the cime; hind dorm,
I cathatess not
Thithers hav' beat lke'n thaing.

BUTUT'MEN:
ARIN
Bi